<a href="https://colab.research.google.com/github/riyakwl28/musix/blob/master/mir_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd /content/drive/My Drive/music_genre

/content/drive/My Drive/music_genre


In [0]:
#feature extractoring and preprocessing data
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

#Keras
import keras

import warnings

In [20]:
data = pd.read_csv('data.csv')
data.head()

,filename,chroma_stft,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,blues.00044.wav,0.390212,2279.124558,2375.102120,5198.360233,0.092570,-109.509285,86.922406,-8.607986,64.494557,...,11.087480,-5.085794,3.976359,-12.859741,12.343857,0.026217,-0.741570,-5.126619,3.303441,blues
1,blues.00010.wav,0.303954,1410.469096,1512.620983,2767.274933,0.062981,-173.707514,137.156400,-23.766517,26.989370,...,1.800535,-2.988374,-1.869389,-0.800155,-3.023782,-0.445196,-1.998228,-0.743599,-4.986112,blues
2,blues.00002.wav,0.363603,1552.481958,1747.165985,3040.514948,0.076301,-90.754394,140.459907,-29.109965,31.689014,...,5.789265,-8.905224,-1.083720,-9.218359,2.455805,-7.726901,-1.815724,-3.433434,-2.226821,blues
3,blues.00075.wav,0.466436,2225.216647,2255.195005,4703.188227,0.108587,-86.352338,92.675245,-15.377407,54.059777,...,12.181273,-6.097625,7.025420,-5.805348,7.432128,-6.075143,3.382991,-4.276934,1.238193,blues
4,blues.00039.wav,0.255321,1151.340441,1709.193938,2312.577683,0.035339,-301.686274,111.539209,21.895657,16.145590,...,-10.572882,-15.687018,-12.598773,-12.245661,-14.543065,-10.092300,-5.780286,-7.776049,-7.186643,blues


In [21]:

data = data.drop(['filename'],axis=1)
data.shape

(1100, 26)

In [32]:
import numpy as np
from keras.utils import to_categorical
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)
len(y_train),len(y_test)

x_val = X_train[:200]
partial_x_train = X_train[200:]

y_val = y_train[:200]
partial_y_train = y_train[200:]

X_train.shape
y_train.shape




(880, 10)

In [34]:
#model def
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers import Dense
from keras.optimizers import Adam

opt = Adam()

batch_size = 35
nb_epochs = 400
timesteps=1
data_dim=25

print('Build LSTM RNN model ...')
model = Sequential()
model.add(LSTM(units=128, dropout=0.05, recurrent_dropout=0.35, return_sequences=True, input_shape=(timesteps,data_dim)))
model.add(LSTM(units=32, dropout=0.05, recurrent_dropout=0.35, return_sequences=False))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.summary()

model.fit(partial_x_train, partial_y_train, batch_size=batch_size, epochs=nb_epochs)


score, accuracy = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print("Dev loss:  ", score)
print("Dev accuracy:  ", accuracy)

score, accuracy = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print("Test loss:  ", score)
print("Test accuracy:  ", accuracy)




Build LSTM RNN model ...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 1, 128)            78848     
_________________________________________________________________
lstm_9 (LSTM)                (None, 32)                20608     
_________________________________________________________________
dense_4 (Dense)              (None, 10)                330       
Total params: 99,786
Trainable params: 99,786
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
Epoch 1/400
680/680 [==============================] - 4s 6ms/step - loss: 2.2741 - acc: 0.2647
Epoch 2/400
680/680 [==============================] - 0s 418us/step - loss: 2.1782 - acc: 0.3676
Epoch 3/400
680/680 [==============================] - 0s 391us/step - loss: 2.0315 - acc: 0.3779
Epoch 4/400
680/680 [================